In [ ]:
# -*- coding: utf-8 -*-
"""volo_petfinder.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1HwTfmBLJHzaCv0O3X4BpGBMJCoiWCkgB
"""


import torch
torch.__version__

import os, sys, gc
sys.path.insert(0,r"F:\Pycharm_projects\PetFinder\models\VOLO/")

import typing as tp
import albumentations as A
from sklearn.metrics import mean_squared_error

import timm
from timm import create_model
#from timm.data.mixup import Mixup

from fastai.vision.all import *
#from fastai.callback.hook import *
from volo.models import volo_d1, volo_d2, volo_d3, volo_d4, volo_d5  # register models to timm
from volo.utils import load_pretrained_weights as volo_load_weights
from fastai.callback.all import *
VOLO_CHECHPOINTS = {
    "volo_d1": "F:\Pycharm_projects\PetFinder\models\VOLO/d1_224_84.2.pth.tar",
    "volo_d1-384": "F:\Pycharm_projects\PetFinder\models\VOLO/d1_384_85.2.pth.tar",
    "volo_d2": "F:\Pycharm_projects\PetFinder\models\VOLO/d2_224_85.2.pth.tar",
    "volo_d2-384": "F:\Pycharm_projects\PetFinder\models\VOLO/d2_384_86.0.pth.tar",
    "volo_d3": "..F:\Pycharm_projects\PetFinder\models\VOLO/d3_224_85.4.pth.tar",
    "volo_d3-448": "F:\Pycharm_projects\PetFinder\models\VOLO/d3_448_86.3.pth.tar",
    "volo_d4": "F:\Pycharm_projects\PetFinder\models\VOLO/d4_224_85.7.pth.tar",
    "volo_d4-448": "F:\Pycharm_projects\PetFinder\models\VOLO/d4_448_86.79.pth.tar",
    "volo_d5": "F:\Pycharm_projects\PetFinder\models\VOLO/d5_224_86.10.pth.tar",
    "volo_d5-448": "F:\Pycharm_projects\PetFinder\models\VOLO/d5_448_87.0.pth.tar",
    "volo_d5-512": "F:\Pycharm_projects\PetFinder\models\VOLO/d5_512_87.07.pth.tar",
}



class BasicImageModel(nn.Module):

    def __init__(self,
                 base_name: str,
                 dims_head: tp.List[int],
                 pretrained=True,
                 in_channels: int=3,
                 image_size: int=224,
                 drop_path_rate: float=0.1
    ):
        """Initialize"""
        self.base_name = base_name
        super().__init__()
        model_name = base_name.split("-")[0]
        assert timm.is_model(model_name), "you can use only models in timm."

        if model_name[:4] == "volo":
            base_model = timm.create_model(
                model_name, img_size=image_size,
                mix_token=False,
                return_dense=False,
                drop_path_rate=0.3, # 0.4,  0.1
            )
            in_features = base_model.head.in_features
            if pretrained:
                volo_load_weights(base_model, VOLO_CHECHPOINTS[base_name], strict=False)

            if in_channels != 3:
                # # change input channel
                # # I follow the manner used in timm.
                first_conv = base_model.patch_embed.conv[0]
                w_t = first_conv.weight.data  # shape: (out_ch, 3, 7, 7)
                if in_channels == 1:
                    new_w_t = w_t.sum(axis=1, keepdims=True)  # shape: (out_ch, 1, 7, 7)
                else:
                    n_repeats = (in_channels + 3 - 1) // 3
                    new_w_t = w_t.repeat((1, n_repeats, 1, 1))
                    new_w_t = new_w_t[:, :in_channels]
                    new_w_t = new_w_t * 3 / in_channels  # shape: (out_ch, in_channels, 7, 7)

                first_conv.weight.data = new_w_t
        else:
            base_model = timm.create_model(
                base_name, pretrained=pretrained, in_chans=in_channels)
            in_features = base_model.num_features
            print("load imagenet pretrained:", pretrained)

        base_model.reset_classifier(num_classes=0)
        self.backbone = base_model
        print(f"{base_name}: {in_features}")

        # # prepare head clasifier
        if dims_head[0] is None:
            dims_head[0] = in_features

        layers_list = []
        for i in range(len(dims_head) - 2):
            in_dim, out_dim = dims_head[i: i + 2]
            layers_list.extend([
                nn.Linear(in_dim, out_dim),
                nn.ReLU(), nn.Dropout(0.5),])
        layers_list.append(
            nn.Linear(dims_head[-2], dims_head[-1]))
        self.head = nn.Sequential(*layers_list)

    def forward(self, x):
        """Forward"""
        h = self.backbone(x)
        h = self.head(h)
        return h

seed=365
set_seed(seed, reproducible=True)
# set_seed(365, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

BATCH_SIZE = 4 # 8
N_EPOCHS = 12 # 6,7
NEED_TRAIN = True

dataset_path = Path(r'F:\Pycharm_projects\PetFinder\data\train')

train_df = pd.read_csv('F:\Pycharm_projects\PetFinder\data/train_5folds.csv')
train_df['path'] = train_df['Id'].map(lambda x: str(dataset_path/x) + '.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

len_df = len(train_df)
print(f"There are {len_df} images")

train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")

print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")

train_df['norm_score'] = train_df['Pawpularity']/100
train_df['norm_score']

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(torch.sigmoid(input.flatten()), target))

#!pip install radam

from radam import Over9000



def WrapperOver9000(param_groups,**kwargs):
    return OptimWrapper(param_groups,Over9000)
project_id = 'volo_d2-384_with_RandomToneCurve'
tboard_path = Path(r'F:\Pycharm_projects\PetFinder\Tensorboard' + project_id)

import random
import cv2
from matplotlib import pyplot as plt

import albumentations as A


def visualize(image):
	plt.figure(figsize=(10, 10))
	plt.axis('off')
	plt.imshow(image)




def train_aug():
	return A.Compose([
		A.Resize(384, 384),
		A.SomeOf([A.RandomContrast((0.6, 0.3)), A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20),
		          A.RandomBrightness(limit=[0.6, 0.3]), A.Flip(), A.Rotate(limit=45), A.Equalize(), A.Sharpen(),
		          A.Solarize(196), A.Posterize(), A.Affine(translate_percent=[0.1, 0.1])], n=3)
	])


def valid_aug():
	return A.Compose([
		A.Resize(384, 384),

	])


class AlbumentationsTransform(RandTransform):
	"A transform handler for multiple `Albumentation` transforms"
	split_idx, order = None, 2

	def __init__(self, train_aug, valid_aug):
		store_attr()

	def before_call(self, b, split_idx):
		self.idx = split_idx

	def encodes(self, img: PILImage):
		if self.idx == 0:
			aug_img = self.train_aug(image=np.array(img))['image']
		else:
			aug_img = self.valid_aug(image=np.array(img))['image']
		return PILImage.create(aug_img)

def get_data(fold):

    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['kfold'] == fold)
    val_idx = train_df_f[train_df_f['is_valid']].index.values

    # blocks=(ImageBlock, CategoryBlock), get_y would be standard Pawpularity
    dls = DataBlock(blocks=(ImageBlock, RegressionBlock),
	                get_x=ColReader('path'),
	                get_y=ColReader('norm_score'),
	                splitter=IndexSplitter(val_idx),
	                item_tfms=[AlbumentationsTransform(train_aug(), valid_aug())],
									        batch_tfms=Normalize.from_stats( [0.5183,  0.4835 ,0.4457],[0.2681 ,0.2638, 0.2658])#pass in item_tfms
	                )
    paw_dls = dls.dataloaders(train_df_f,
	                          bs=BATCH_SIZE,
	                          num_workers=0,
	                          seed=seed)



    return paw_dls#, splitter

def get_learner(fold_num):
    data = get_data(fold_num)

    #model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)
    model = BasicImageModel(base_name='volo_d2-384',
                            dims_head=[None, 1],
                            image_size=384,
                            drop_path_rate=0.3)

    # alternatives in cbs for regularization: CutMix(1.0), MixUp(0.4) - use only one or the other
    learn = Learner(data,
                    model,
                    loss_func=BCEWithLogitsLossFlat(),
                    metrics=AccumMetric(petfinder_rmse),
                    cbs=[MixUp(1.0),RandTransform()],
                    opt_func=Adam
                    #opt_func=ranger,
                    #opt_func=partial(WrapperOver9000, eps=1e-4)
                   ).to_fp16()


    return learn #, splitter

auglist = [A.RandomBrightness(),A.RandomGamma(),A.RandomBrightnessContrast()]

dls = get_data(0)
dls.show_batch()

class RandAug(Transform):
    def __init__(self,n,m,auglist):
        self.auglist = auglist
        self.n = n
        self.m = m
    def encode(self,img: PILImage):
        img = np.array(img)
        ops = random.choices(self.auglist,k=self.n)
        for op,minval,maxval in ops:
            val = (float(self.m) / 30) * float(maxval - minval) + minval
            img = op(img,val)['image']
        return PILImage.create(img)

train_df['norm_score'] = train_df['Pawpularity']/100

if NEED_TRAIN:
    rmse_list = []
    ##for i in range(5):
    for i in [0,4]:

        print(f'Fold {i} results')

        learn = get_learner(fold_num=i)
        lr = learn.lr_find(end_lr=3e-2)
        print("training")

        print("lr")

        learn.fit_one_cycle(N_EPOCHS, 2e-5/2, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)])
        learn.recorder.plot_loss()

        #over fitting
        # learn.unfreeze()

        # learn.fit_one_cycle(5,lr_max=slice(1e-6,1e-4))
        #learn = learn.to_fp32()

        learn.export(f'F:\Pycharm_projects\PetFinder\models/model_fold_{i}.pkl')

        #all_preds.append(preds)

        val_idx = train_df[train_df['kfold'] == i].index
        val_df = train_df.loc[val_idx]

        val_pred, _ = learn.get_preds()
        val_pred = val_pred.numpy()
        val_pred = val_pred * 100

        train_df.loc[val_idx, 'oof'] = val_pred
        print('Pawpularity vs OOF')
        print(val_df['Pawpularity'][:5], val_pred[:5])

        real_rmse = mean_squared_error(train_df[train_df['kfold'] == i]['Pawpularity'],
                    train_df[train_df['kfold'] == i]['oof'], squared=False)

        fold_rmse = mean_squared_error(train_df[train_df['kfold'] == i]['oof'], train_df[train_df['kfold'] == i]['Pawpularity'], squared=False)
        print(f'Fold {i} RMSE: {fold_rmse}')
        rmse_list.append(fold_rmse)


        del learn
        torch.cuda.empty_cache()
        gc.collect();

print(f'Average Fold RMSE: {sum(rmse_list) / len(rmse_list)}')



train_df[['oof','Pawpularity']].plot.scatter('oof','Pawpularity')

train_df[['oof','Pawpularity']].hist()

train_df.to_csv(f'train_with_oof_seed_{seed}.csv')



There are 9885 images
The mean Pawpularity score is 38.049974709155286
The median Pawpularity score is 33.0
The standard deviation of the Pawpularity score is 20.599054723289793
There are 100 unique values of Pawpularity score


F:\anaconda\envs\tensorflow2\lib\site-packages\albumentations\augmentations\transforms.py:1701: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
F:\anaconda\envs\tensorflow2\lib\site-packages\albumentations\augmentations\transforms.py:1727: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


Fold 0 results


F:\anaconda\envs\tensorflow2\lib\site-packages\albumentations\augmentations\transforms.py:1727: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
F:\anaconda\envs\tensorflow2\lib\site-packages\albumentations\augmentations\transforms.py:1701: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


volo_d2-384: 512


training
lr


epoch,train_loss,valid_loss,petfinder_rmse,time


In [ ]:
print("lr")

In [ ]:
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
print(servers)